In [20]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import load_img, img_to_array


In [21]:
# Step 1: Load and prepare your dataset
def load_dates_images(image_dir, target_size=(64, 64)):
    images = []
    for filename in os.listdir(image_dir):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            img_path = os.path.join(image_dir, filename)
            img = load_img(img_path, target_size=target_size, color_mode='rgb')
            img = img_to_array(img) / 255.0  # Normalize to [0, 1]
            images.append(img)
    return np.array(images)

# Load images
X_train = load_dates_images('/Users/aiwaziri/PycharmProjects/GitHub/Madinah_dates_hackathon/advanced_augumentation/augumented_testing')

In [22]:
X_train.shape

(70, 64, 64, 3)

In [23]:
# Step 2: Build the Generator
def build_generator():
    model = tf.keras.Sequential([
        layers.Dense(128 * 16 * 16, activation='relu', input_shape=(100,)),
        layers.Reshape((16, 16, 128)),
        layers.Conv2DTranspose(128, kernel_size=3, strides=2, padding='same', activation='relu'),
        layers.Conv2DTranspose(64, kernel_size=3, strides=2, padding='same', activation='relu'),
        layers.Conv2D(3, kernel_size=3, padding='same', activation='tanh')  # For RGB images
    ])
    return model

In [24]:
# Step 3: Build the Discriminator
def build_discriminator():
    model = tf.keras.Sequential([
        layers.Conv2D(64, kernel_size=3, strides=2, padding='same', input_shape=(64, 64, 3)),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(128, kernel_size=3, strides=2, padding='same'),
        layers.LeakyReLU(alpha=0.2),
        layers.Flatten(),
        layers.Dense(1, activation='sigmoid')
    ])
    return model

In [25]:
# Step 4: Compile the GAN
generator = build_generator()
discriminator = build_discriminator()
discriminator.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Create the GAN model
discriminator.trainable = False
z = layers.Input(shape=(100,))
img = generator(z)
validity = discriminator(img)
gan = tf.keras.Model(z, validity)
gan.compile(optimizer='adam', loss='binary_crossentropy')


In [26]:
# Function to sample images
def sample_images(epoch):
    noise = np.random.normal(0, 1, (25, 100))
    generated_imgs = generator.predict(noise)
    generated_imgs = 0.5 * generated_imgs + 0.5  # Rescale to [0, 1]

    plt.figure(figsize=(5, 5))
    for i in range(generated_imgs.shape[0]):
        plt.subplot(5, 5, i + 1)
        plt.imshow(generated_imgs[i, :, :, :])
        plt.axis('off')
    plt.savefig(f"gan_generated_epoch_{epoch}.png")
    plt.close()


In [27]:
# Step 5: Training function
def train_gan(epochs, batch_size):
    for epoch in range(epochs):
        # Train the discriminator
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        real_imgs = X_train[idx]

        noise = np.random.normal(0, 1, (batch_size, 100))
        fake_imgs = generator.predict(noise)

        d_loss_real = discriminator.train_on_batch(real_imgs, np.ones((batch_size, 1)))
        d_loss_fake = discriminator.train_on_batch(fake_imgs, np.zeros((batch_size, 1)))
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train the generator
        noise = np.random.normal(0, 1, (batch_size, 100))
        g_loss = gan.train_on_batch(noise, np.ones((batch_size, 1)))

        # Print the progress
        if epoch % 100 == 0:
            print(f"{epoch} [D loss: {d_loss[0]} | D accuracy: {100 * d_loss[1]:.2f}] [G loss: {g_loss}]")
            sample_images(epoch)


In [29]:
# Step 6: Start training
train_gan(epochs=100, batch_size=64)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
0 [D loss: 1.5060129165649414 | D accuracy: 3.70] [G loss: [array(1.5178189, dtype=float32), array(1.5178189, dtype=float32), array(0.03681144, dtype=float32)]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
2/2